In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# Importing Library:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from keras.layers import Conv2D, MaxPooling2D


In [3]:
# Dataset Path:
BATCH_SIZE = 64
IMAGE_SIZE = 300
train_path = "/content/drive/MyDrive/Major Project/Skin disease/train"
test_path = "/content/drive/MyDrive/Major Project/Skin disease/test"

In [4]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR,IMAGE_SIZE,BATCH_SIZE):
  train_datagen = ImageDataGenerator(rescale=(1./255),
                                     shear_range=0.2,
                                     zoom_range=0.3,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     brightness_range=[0.2,1.2],
                                     rotation_range=0.2,
                                     horizontal_flip=True)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=BATCH_SIZE,
                                                      class_mode='categorical',
                                                      target_size=(IMAGE_SIZE, IMAGE_SIZE))

  test_datagen = ImageDataGenerator(rescale=1./255)


  test_generator = test_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE))

  return train_generator, test_generator

In [6]:
train_generator,test_generator=train_val_generators(train_path, test_path,IMAGE_SIZE,BATCH_SIZE)

FileNotFoundError: ignored

In [ ]:
 for i in range(5):
     img, label = train_generator.next()
     print(img.shape)
     print(label[0])
     plt.imshow(img[0])
     plt.show()

In [ ]:
class_names = train_generator.class_indices
class_names

In [ ]:
 from tensorflow.keras.applications.inception_v3 import InceptionV3
 base_model = InceptionV3(input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3), include_top = False, weights = 'imagenet')

In [ ]:
 for layer in base_model.layers:
     layer.trainable = True

In [ ]:
 def output_of_last_layer(pre_trained_model, limit_layer):
   last_desired_layer = pre_trained_model.get_layer(limit_layer)
   print('last layer output shape: ', last_desired_layer.output_shape)
   last_output = last_desired_layer.output
   print('last layer output: ', last_output)

   return last_output

In [ ]:
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # Create a sequential model
# model = Sequential()

# # Add the first convolutional layer with 32 filters, a 3x3 kernel, and ReLU activation
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(300,300,3)))

# # Add the first pooling layer with 2x2 pool size
# model.add(MaxPooling2D((2, 2)))

# # Add a second convolutional layer with 64 filters, a 3x3 kernel, and ReLU activation
# model.add(Conv2D(64, (3, 3), activation='relu'))

# # Add a second pooling layer with 2x2 pool size
# model.add(MaxPooling2D((2, 2)))

# # Add a third convolutional layer with 128 filters, a 3x3 kernel, and ReLU activation
# model.add(Conv2D(128, (3, 3), activation='relu'))

# # Add a third pooling layer with 2x2 pool size
# model.add(MaxPooling2D((2, 2)))

# # Flatten the output of the convolutional layers
# model.add(Flatten())

# # Add a fully connected layer with 512 neurons and ReLU activation
# model.add(Dense(512, activation='relu'))

# # Add dropout regularization
# model.add(Dropout(0.5))

# # Add a final softmax layer for classification
# model.add(Dense(23, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import regularizers
# from keras.layers import Conv2D, MaxPooling2D

# x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), kernel_regularizer=regularizers.l2(0.01))(last_output)
# x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
# # Commenting out the following MaxPooling2D layer
# # x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3))(x)
# x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
# x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(23, activation='softmax')(x)

# model = tf.keras.models.Model(base_model.input, x)

# model.compile(
#     optimizer = Adam(learning_rate=0.001),
#     loss = 'categorical_crossentropy',
#     metrics = ['accuracy']
# )


In [ ]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = True

# Add new layers
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(23, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    min_delta=0.001,
    mode='min'
)

*Using Xception model*

In [ ]:
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

# Parameters
img_width, img_height = 299, 299
train_data_dir = "/content/drive/MyDrive/Major Project/Skin disease/train"
validation_data_dir = "/content/drive/MyDrive/Major Project/Skin disease/test"
batch_size = 32
epochs = 10
num_classes = 23

# Load the pre-trained Xception model
base_model = Xception(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Save the model
model.save('xception_transfer_learning.h5')


83683744/83683744 [==============================] - 0s 0us/step


FileNotFoundError: ignored

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure(figsize=(6,6))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure(figsize=(6,6))


plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import numpy as np

test_true=test_generator.classes
test_pred_raw = model.predict(test_generator)
test_pred = np.argmax(test_pred_raw, axis=1)

cm = confusion_matrix(test_true, test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig, ax = plt.subplots(figsize=(15,15))
disp.plot(ax=ax,cmap=plt.cm.Blues)
plt.show()

In [ ]:
# !mkdir -p saved_model
model.save('my_model')